In [33]:
import os
import numpy as np
import pandas as pd
import openpyxl
from openpyxl import load_workbook
 
data_folder = os.path.join(os.path.dirname(os.getcwd())+'/data/')

extracted_data = {}

for file in os.listdir(data_folder):
    filepath = os.path.join(data_folder, file)
    dataframe = pd.read_excel(filepath, engine = 'openpyxl')
    extracted_data[file] = dataframe

# for name, data in extracted_data.items():
#     print(f"{name}:{data.shape}")

trip_advisor_data = extracted_data["trip_advisor_v2.xlsx"]
yelp_data = extracted_data["yelp_data_scraped2023.xlsx"]

combined_df = pd.merge(trip_advisor_data, yelp_data, left_on=['Restaurant'], right_on=['Name'], how='outer')

print(yelp_data.shape[0] + trip_advisor_data.shape[0] - combined_df.shape[0])

41
